In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
# autoreloads imports every time a module is called

In [97]:
from transformers import get_transformer

In [98]:
models, tokenizer = get_transformer('gpt2')

In [99]:
encoder, decoder = models

In [100]:
sent1 = "my name is [SOMEONE], and you killed my father"
sent2 = "do not go gentle into that good night"

In [101]:
import torch
from transformers import make_batch

x, lengths = make_batch(tokenizer, [sent1, sent2])

In [102]:
x

tensor([[ 1820,  1438,   318, 50259,    11,   290,   345,  2923,   616,  2988],
        [ 4598,   407,   467, 10296,   656,   326,   922,  1755, 50263, 50263]])

In [103]:
from munch import Munch

args = Munch({
    'keyword_threshold': 1/3,
    'transformer_pool': 'mean', 
    'threshold_gap': 3,
})

In [104]:
from encoder import Encoder

ae_encoder = Encoder(args, encoder, tokenizer)

In [105]:
keywords, keyword_lengths, scores, reg_loss = ae_encoder(x, lengths)

In [159]:
from decoder import Decoder

ae_decoder = Decoder(args, decoder, tokenizer)

In [161]:
logits = ae_decoder(x, keywords, keyword_lengths, scores)

torch.float32


In [162]:
logits.shape

torch.Size([2, 11, 50264])